In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
def set_timestamp(df):
    dia = []
    mes = []
    ano = []
    hora = []
    minuto = []
    segundo = []

    for d, h in zip(df.Data, df.Hora):
        dia.append(int(d[:2]))
        mes.append(int(d[3:5]))
        ano.append(int(d[6:]))
    
        hora.append(int(h[:2]))
        minuto.append(int(h[3:5]))
        segundo.append(int(h[6:]))

    

    data = pd.DataFrame({'year': ano,
                   'month': mes,
                   'day': dia,
                   'hour': hora,
                   'minute': minuto,
                   'second': segundo})
    index = pd.to_datetime(data)

    df['timer'] = index
    df = df.drop_duplicates(subset=['timer'])
    df.index = df['timer']
    return df

In [ ]:
def group_data(df, frequency = '30min'):    
    columns = ['Temp', 'OD', 'PH', 'NH3', 'AT']
    data = pd.DataFrame()    
    for c in columns:
        group = df.groupby([pd.Grouper(freq=frequency), 'link'], sort=False)[c].mean().unstack().reset_index(
                        ).set_index('timer').resample(frequency).mean().transform(
                        lambda x: x.fillna(method='ffill')).transform(
                        lambda x: x.fillna(method='backfill')).transform(
                        lambda x: x.fillna(method='pad')).dropna()
        
        col = group.values.reshape(1,-1)
        data[c] = col[0]
    
    data.index = group.index
    return data

In [ ]:
def group_by_std(df, frequency = '30min'):    
    columns = ['Temp', 'OD', 'PH', 'NH3', 'AT']
    data = pd.DataFrame()    
    for c in columns:
        group = df.groupby([pd.Grouper(freq=frequency), 'link'], sort=False)[c].std()
        
        col = group.values.reshape(1,-1)
        data[c] = col[0]
    
    data.index = group.index
    return data

In [ ]:
def group_column(dataset, frequency, column):
    dataset = dataset.groupby([pd.Grouper(freq=frequency)], sort=False)[column]

    lista = list(dataset)
    dictionary = {}
    for k, v in lista:  
        dictionary[str(k)] = v
    return dictionary

In [ ]:
# def get_group(dictionary, arg, minute):
#     for k in dictionary:
#         if pd.Timestamp(k) <= arg < pd.Timestamp(k) + pd.Timedelta(minutes=minute):    
#             break
#     return dictionary[k]
def get_group(dictionary, arg, minute):
    keys = []
    for k in dictionary:
        if arg <= pd.Timestamp(k) < arg + pd.Timedelta(minutes=minute):    
            keys.append(k)
        
        if not (arg <= pd.Timestamp(k) < arg + pd.Timedelta(minutes=minute)) and len(keys) > 0:
            break
    
    values = []
    import numpy as np
    for k in keys:
        values = np.concatenate([values, dictionary[k]])
    return np.mean(values)

In [ ]:
def get_group_G(dictionary, arg, minute):
    keys = []
    for k in dictionary:
        if pd.Timestamp(k) <= arg < pd.Timestamp(k) + pd.Timedelta(minutes=minute):    
            keys.append(k)
        
        if not (pd.Timestamp(k) <= arg < pd.Timestamp(k) + pd.Timedelta(minutes=minute)) and len(keys) > 0:
            break
    
    values = []
    import numpy as np
    for k in keys:
        values = np.concatenate([values, dictionary[k]])
    return np.mean(values)

In [ ]:
def train_test_split(df, p):
    X = df.drop(["y"], axis=1)
    y = df.y

    n = int(X.shape[0]*p)

    X_train = X[0:n]
    y_train = y[0:n]

    X_test = X[n:]
    y_test = y[n:]

    X = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    return X, X_test, y, y_test

In [ ]:
def standard(X, X_test):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X_test = scaler.transform(X_test)
    return X, X_test

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):    
    import numpy as np
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
def metrics(y_val, y_pred):
    from sklearn.metrics import mean_absolute_error as mae
    from sklearn.metrics import mean_squared_error as mse

    MAE = mae(y_val, y_pred)
    MSE = mse(y_val, y_pred)
    RMSE = mse(y_val, y_pred, squared=False)  
    MAPE = mean_absolute_percentage_error(y_val, y_pred)

    return {"MAE": MAE, "MSE": MSE, "RMSE": RMSE, "MAPE": MAPE}

In [ ]:
def window_all(df, n, y):
    cols = list(df.columns)*n
    cols.append(y)

    df_new = pd.DataFrame(columns=cols)

    for i in range(0, df.shape[0]-n):
        lista = []

        for j in range(i, i+n):
            lista.append(df.loc[[j]].reset_index(drop=True))

        lista.append(df.loc[[i+n]][y].reset_index(drop=True))  
        df_new = df_new.append(pd.concat(lista, axis=1), ignore_index=True)    

    cols = ['C'+str(i) for i in range(len(cols)-1)]
    cols.append('y')
    df_new.columns = cols  
    return df_new

In [ ]:
df_start = pd.read_csv('dadosfisicosquimicos.csv', sep=';')
df_start = df_start.drop(['Unnamed: 7'], axis=1)
df_start

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(figsize=(18,8))
plt.plot(df_start.Temp, color='#069AF3', label='Temperatura')
plt.xticks(rotation=45)
plt.xlabel('Data')
plt.ylabel('Grau Celsius')
plt.title('Coleta sensorial da temperatura')
plt.legend()

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(figsize=(18,8))
plt.plot(df_start.OD, color='#069AF3', label='Oxigênio dissolvido')
plt.xticks(rotation=45)
plt.xlabel('Data')
plt.ylabel('mg/l')
plt.title('Coleta sensorial do OD')
plt.legend()

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(figsize=(18,8))
plt.plot(df_start.PH, color='#069AF3', label='Potencial hidrogeniônico')
plt.xticks(rotation=45)
plt.xlabel('Data')
plt.ylabel('pH')
plt.title('Coleta sensorial do pH')
plt.legend()

In [ ]:
df_start['link'] = ['x']*df_start.shape[0]
df_start = set_timestamp(df_start)

In [ ]:
df_std = group_by_std(df_start, '30min')
df_std

In [ ]:
import matplotlib.pyplot as plt

data = [df_std.Temp, df_std.OD, df_std.PH]
labels = ['Temperatura', 'OD', 'pH']

plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(figsize=(18,8))

bplot = ax.boxplot(data, patch_artist=True, labels= labels, widths=0.45, medianprops={"linewidth": 2})


# fill with colors
colors = ['pink', 'lightblue', 'lightgreen']
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)

# adding horizontal grid lines
ax.yaxis.grid(True)
ax.set_xlabel('Sensor')
ax.set_ylabel('Desvio padrão')
plt.title('Desvio padrão resultante do agrupamento de 30 minutos.')
plt.show()

In [ ]:
df = group_data(df_start)
df

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(figsize=(18,8))
plt.plot(df.Temp, color='#069AF3', label='Temperatura')
plt.xticks(rotation=45)
plt.xlabel('Data')
plt.ylabel('Grau Celsius')
plt.title('Coleta sensorial da temperatura')
plt.legend()

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(figsize=(18,8))
plt.plot(df.OD, color='#069AF3', label='Oxigênio dissolvido')
plt.xticks(rotation=45)
plt.xlabel('Data')
plt.ylabel('mg/l')
plt.title('Coleta sensorial do OD')
plt.legend()

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(figsize=(18,8))
plt.plot(df.PH, color='#069AF3', label='Potencial hidrogeniônico')
plt.xticks(rotation=45)
plt.xlabel('Data')
plt.ylabel('pH')
plt.title('Coleta sensorial do pH')
plt.legend()

In [ ]:
OD_40 = group_column(df_start, '40min', 'OD')
OD_50 = group_column(df_start, '50min', 'OD')
OD_60 = group_column(df_start, '60min', 'OD')

PH_40 = group_column(df_start, '40min', 'PH')
PH_50 = group_column(df_start, '50min', 'PH')
PH_60 = group_column(df_start, '60min', 'PH')

Temp_40 = group_column(df_start, '40min', 'Temp')
Temp_50 = group_column(df_start, '50min', 'Temp')
Temp_60 = group_column(df_start, '60min', 'Temp')

In [ ]:
new = pd.DataFrame(columns=['OD_40', 'OD_50', 'OD_60',
                            'PH_40', 'PH_50', 'PH_60',
                            'TEMP_40', 'TEMP_50', 'TEMP_60'])

for value, idx in zip(df.index, range(df.shape[0])):

    ODm_40 = get_group_G(OD_40, value, 40)
    ODm_50 = get_group_G(OD_50, value, 50)
    ODm_60 = get_group_G(OD_60, value, 60)

    PHm_40 = get_group_G(PH_40, value, 40)
    PHm_50 = get_group_G(PH_50, value, 50)
    PHm_60 = get_group_G(PH_60, value, 60)

    Tempm_40 = get_group_G(Temp_40, value, 40)
    Tempm_50 = get_group_G(Temp_50, value, 50)
    Tempm_60 = get_group_G(Temp_60, value, 60)
    
    new.loc[idx] = [
                    ODm_40, ODm_50, ODm_60,
                    PHm_40, PHm_50, PHm_60,
                    Tempm_40, Tempm_50, Tempm_60
                   ]

new

In [ ]:
groupedData = df.reset_index(drop=True)
new = new.reset_index(drop=True)

result = pd.concat([groupedData, new], axis=1)
result

In [ ]:
result.to_csv('dataset_preprocessado.csv')


In [ ]:
result = pd.read_csv('dataset_preprocessado.csv', sep=',')
result = result.drop(['Unnamed: 0'], axis=1)
result

In [ ]:
def baseline(feature):
    column = [feature]
    column_y = feature
    window = 1
    train_size = 0.7
    shift_prediction = 6
    
    data = window_all(result[column].reset_index(drop=True), window, column_y)
    data = data.dropna()
    
    shift_prediction *= -1
    X = data.drop(['y'], axis=1)
    y = data['y'].shift(periods=shift_prediction)
    data = pd.concat([X, y], axis=1)
    data = data[:shift_prediction]
    data = data.astype('float64')
    X_train, X_test, y_train, y_test = train_test_split(data, train_size)
    

    from sklearn.linear_model import LinearRegression

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return metrics(y_test, y_pred)

In [ ]:
def baseline_error(feature):
    column = [feature]
    column_y = feature
    window = 1
    train_size = 0.7
    shift_prediction = 6
    
    data = window_all(result[column].reset_index(drop=True), window, column_y)
    data = data.dropna()
    
    shift_prediction *= -1
    X = data.drop(['y'], axis=1)
    y = data['y'].shift(periods=shift_prediction)
    data = pd.concat([X, y], axis=1)
    data = data[:shift_prediction]
    data = data.astype('float64')
    X_train, X_test, y_train, y_test = train_test_split(data, train_size)
    

    from sklearn.linear_model import LinearRegression

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_test-y_pred

In [ ]:
def treino_neuronios(feature, saida):
    column = [feature]
    column_y = feature
    window = 1
    train_size = 0.7
    shift_prediction = 6
    
    data = window_all(result[column].reset_index(drop=True), window, column_y)
    data = data.dropna()
    
    shift_prediction *= -1
    X = data.drop(['y'], axis=1)
    y = data['y'].shift(periods=shift_prediction)
    data = pd.concat([X, y], axis=1)
    data = data[:shift_prediction]
    data = data.astype('float64')
    X_train, X_test, y_train, y_test = train_test_split(data, train_size)

    
    from sklearn.neural_network import MLPRegressor
    
    df_saida = pd.DataFrame(columns=["neurons", "itaration", "MAE", "MSE", "RMSE", "MAPE"])
    df_saida.to_csv(saida, mode='a')

    
    for n in range(20,220, 20):
        
        df_saida = pd.DataFrame(columns=["neurons", "itaration", "MAE", "MSE", "RMSE", "MAPE"])
        
        for i in range(1, 101):
            model = MLPRegressor(hidden_layer_sizes=(n), max_iter=1000, solver='adam',
                         validation_fraction=0.1,  verbose=False)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            M = metrics(y_test.values, y_pred)
            M['neurons'] = n
            M['itaration'] = i
            df_saida = df_saida.append(M, ignore_index=True)
    
        df_saida.to_csv(saida, mode='a', header=False)
    
    return df_saida

In [ ]:
def treino_camadas(feature, neuronios, saida):
    column = [feature]
    column_y = feature
    window = 1
    train_size = 0.7
    shift_prediction = 6
    
    data = window_all(result[column].reset_index(drop=True), window, column_y)
    data = data.dropna()
    
    shift_prediction *= -1
    X = data.drop(['y'], axis=1)
    y = data['y'].shift(periods=shift_prediction)
    data = pd.concat([X, y], axis=1)
    data = data[:shift_prediction]
    data = data.astype('float64')
    X_train, X_test, y_train, y_test = train_test_split(data, train_size)

    
    from sklearn.neural_network import MLPRegressor
    
    df_saida = pd.DataFrame(columns=["layers", "itaration", "MAE", "MSE", "RMSE", "MAPE"])
    df_saida.to_csv(saida, mode='a')

    neurons = []
    
    for n in range(2,6):
        
        df_saida = pd.DataFrame(columns=["layers", "itaration", "MAE", "MSE", "RMSE", "MAPE"])
        neurons.append(neuronios)
        
        for i in range(1, 101):
            model = MLPRegressor(hidden_layer_sizes=tuple(neurons), max_iter=1000, solver='adam',
                         validation_fraction=0.1,  verbose=False)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            M = metrics(y_test.values, y_pred)
            M['layers'] = n
            M['itaration'] = i
            df_saida = df_saida.append(M, ignore_index=True)
    
        df_saida.to_csv(saida, mode='a', header=False)
    
    return df_saida

In [ ]:
def treino_features(features, y, neuronios, saida):
    column = []
    column_y = y
    window = 1
    train_size = 0.7
    shift_prediction = 6
    
    df_saida = pd.DataFrame(columns=["n_features", "itaration", "MAE", "MSE", "RMSE", "MAPE"])
    df_saida.to_csv(saida, mode='a')
        
    for c in features:
        column.append(c)
        data = window_all(result[column].reset_index(drop=True), window, column_y)
        data = data.dropna()
        data = data.reset_index(drop=True)
        
        shift_prediction = 6
        shift_prediction *= -1
        X = data.drop(['y'], axis=1)
        y = data['y'].shift(periods=shift_prediction)
        data = pd.concat([X, y], axis=1)
        data = data[:shift_prediction]
        data = data.astype('float64')
        
        X_train, X_test, y_train, y_test = train_test_split(data, train_size)

        from sklearn.neural_network import MLPRegressor
            
        
        df_saida = pd.DataFrame(columns=["n_features", "itaration", "MAE", "MSE", "RMSE", "MAPE"])
        
        for i in range(1, 101):
            model = MLPRegressor(hidden_layer_sizes=neuronios, max_iter=1000, solver='adam',
                         validation_fraction=0.1,  verbose=False)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            M = metrics(y_test.values, y_pred)
            M['n_features'] = len(column)
            M['itaration'] = i
            df_saida = df_saida.append(M, ignore_index=True)
    
        df_saida.to_csv(saida, mode='a', header=False)
    
    return df_saida

In [ ]:
def treino_janelas(features, y, neuronios, saida):
    column = features
    column_y = y
    train_size = 0.7
    shift_prediction = 6
    
    df_saida = pd.DataFrame(columns=["windows", "itaration", "MAE", "MSE", "RMSE", "MAPE"])
    df_saida.to_csv(saida, mode='a')
        
    for window in range(1,6):
        
        data = window_all(result[column].reset_index(drop=True), window, column_y)
        data = data.dropna()
        data = data.reset_index(drop=True)
        
        shift_prediction = 6
        shift_prediction *= -1
        X = data.drop(['y'], axis=1)
        y = data['y'].shift(periods=shift_prediction)
        data = pd.concat([X, y], axis=1)
        data = data[:shift_prediction]
        data = data.astype('float64')
        
        X_train, X_test, y_train, y_test = train_test_split(data, train_size)

        from sklearn.neural_network import MLPRegressor
            
        
        df_saida = pd.DataFrame(columns=["windows", "itaration", "MAE", "MSE", "RMSE", "MAPE"])
        
        for i in range(1, 101):
            model = MLPRegressor(hidden_layer_sizes=neuronios, max_iter=1000, solver='adam',
                         validation_fraction=0.1,  verbose=False)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            M = metrics(y_test.values, y_pred)
            M['windows'] = window
            M['itaration'] = i
            df_saida = df_saida.append(M, ignore_index=True)
    
        df_saida.to_csv(saida, mode='a', header=False)
    
    return df_saida

In [ ]:
def analise_neuronios(arquivo):
    df = pd.read_csv(arquivo)
    df = df.drop(columns=['Unnamed: 0'])
    
    M_MAE = []
    S_MAE = []

    M_RMSE = []
    S_RMSE = []

    M_MAPE = []
    S_MAPE = []
    
    for n in range(20,220,20):
        
        lista = df.loc[df.neurons == n]
    
        M_MAE.append(lista.MAE.mean())
        S_MAE.append(lista.MAE.std())
    
        M_RMSE.append(lista.RMSE.mean())
        S_RMSE.append(lista.RMSE.std())
    
        M_MAPE.append(lista.MAPE.mean())
        S_MAPE.append(lista.MAPE.std())
    
    import numpy as np
    M_MAE = np.array(M_MAE)
    S_MAE = np.array(S_MAE)

    M_RMSE = np.array(M_RMSE)
    S_RMSE = np.array(S_RMSE)

    M_MAPE = np.array(M_MAPE)
    S_MAPE = np.array(S_MAPE)

    import matplotlib.pyplot as plt
    X = list(range(20,220,20))

    fig, ax = plt.subplots(figsize=(13.7, 8.27))

    plt.plot(X, M_RMSE, color='purple', marker='o', markersize=5, label='RMSE')
    plt.fill_between(X, M_RMSE + S_RMSE, M_RMSE - S_RMSE, alpha=0.15, color='purple')

    plt.plot(X, M_MAE, color='green', marker='o', markersize=5, label='MAE')
    plt.fill_between(X, M_MAE + S_MAE, M_MAE - S_MAE, alpha=0.15, color='green')

    plt.plot(X, M_MAPE, color='gold', marker='o', markersize=5, label='MAPE')
    plt.fill_between(X, M_MAPE + S_MAPE, M_MAPE - S_MAPE, alpha=0.15, color='gold')
    

    plt.grid()
    plt.title("Erro médio dos 100 treinamentos de cada arquitetura MLP", fontsize=18)
    plt.xlabel('Número de neurônios', fontsize=18)
    plt.ylabel('Erro', fontsize=18)
    plt.legend(fontsize=18)

In [ ]:
def analise_camadas(arquivo):
    df = pd.read_csv(arquivo)
    df = df.drop(columns=['Unnamed: 0'])
    
    M_MAE = []
    S_MAE = []

    M_RMSE = []
    S_RMSE = []

    M_MAPE = []
    S_MAPE = []
    
    for n in range(2,6):
        
        lista = df.loc[df.layers == n]
    
        M_MAE.append(lista.MAE.mean())
        S_MAE.append(lista.MAE.std())
    
        M_RMSE.append(lista.RMSE.mean())
        S_RMSE.append(lista.RMSE.std())
    
        M_MAPE.append(lista.MAPE.mean())
        S_MAPE.append(lista.MAPE.std())
    
    import numpy as np
    M_MAE = np.array(M_MAE)
    S_MAE = np.array(S_MAE)

    M_RMSE = np.array(M_RMSE)
    S_RMSE = np.array(S_RMSE)

    M_MAPE = np.array(M_MAPE)
    S_MAPE = np.array(S_MAPE)

    import matplotlib.pyplot as plt
    X = list(range(2,6))

    fig, ax = plt.subplots(figsize=(13.7, 8.27))

    plt.plot(X, M_RMSE, color='purple', marker='o', markersize=5, label='RMSE')
    plt.fill_between(X, M_RMSE + S_RMSE, M_RMSE - S_RMSE, alpha=0.15, color='purple')

    plt.plot(X, M_MAE, color='green', marker='o', markersize=5, label='MAE')
    plt.fill_between(X, M_MAE + S_MAE, M_MAE - S_MAE, alpha=0.15, color='green')

    plt.plot(X, M_MAPE, color='gold', marker='o', markersize=5, label='MAPE')
    plt.fill_between(X, M_MAPE + S_MAPE, M_MAPE - S_MAPE, alpha=0.15, color='gold')
    

    plt.grid()
    plt.title("Erro médio dos 100 treinamentos de cada arquitetura MLP", fontsize=18)
    plt.xlabel('Número de neurônios', fontsize=18)
    plt.ylabel('Erro', fontsize=18)
    plt.legend(fontsize=18)

In [ ]:
def analise_derivacao(arquivo):
    df = pd.read_csv(arquivo)
    df = df.drop(columns=['Unnamed: 0'])
    
    M_MAE = []
    S_MAE = []

    M_RMSE = []
    S_RMSE = []

    M_MAPE = []
    S_MAPE = []
    
    for n in range(1,5):
        
        lista = df.loc[df.n_features == n]
    
        M_MAE.append(lista.MAE.mean())
        S_MAE.append(lista.MAE.std())
    
        M_RMSE.append(lista.RMSE.mean())
        S_RMSE.append(lista.RMSE.std())
    
        M_MAPE.append(lista.MAPE.mean())
        S_MAPE.append(lista.MAPE.std())
    
    import numpy as np
    M_MAE = np.array(M_MAE)
    S_MAE = np.array(S_MAE)

    M_RMSE = np.array(M_RMSE)
    S_RMSE = np.array(S_RMSE)

    M_MAPE = np.array(M_MAPE)
    S_MAPE = np.array(S_MAPE)

    import matplotlib.pyplot as plt
    X = list(range(1,5))

    fig, ax = plt.subplots(figsize=(13.7, 8.27))

    plt.plot(X, M_RMSE, color='purple', marker='o', markersize=5, label='RMSE')
    plt.fill_between(X, M_RMSE + S_RMSE, M_RMSE - S_RMSE, alpha=0.15, color='purple')

    plt.plot(X, M_MAE, color='green', marker='o', markersize=5, label='MAE')
    plt.fill_between(X, M_MAE + S_MAE, M_MAE - S_MAE, alpha=0.15, color='green')

    plt.plot(X, M_MAPE, color='gold', marker='o', markersize=5, label='MAPE')
    plt.fill_between(X, M_MAPE + S_MAPE, M_MAPE - S_MAPE, alpha=0.15, color='gold')

    plt.grid()
    plt.title("Erro médio dos 100 treinamentos de cada arquitetura MLP", fontsize=18)
    plt.xlabel('Número de neurônios', fontsize=18)
    plt.ylabel('Erro', fontsize=18)
    plt.legend(fontsize=18)

In [ ]:
def analise_janelas(arquivo):
    df = pd.read_csv(arquivo)
    df = df.drop(columns=['Unnamed: 0'])
    
    M_MAE = []
    S_MAE = []

    M_RMSE = []
    S_RMSE = []

    M_MAPE = []
    S_MAPE = []
    
    for n in range(1,6):
        
        lista = df.loc[df.windows == n]
    
        M_MAE.append(lista.MAE.mean())
        S_MAE.append(lista.MAE.std())
    
        M_RMSE.append(lista.RMSE.mean())
        S_RMSE.append(lista.RMSE.std())
    
        M_MAPE.append(lista.MAPE.mean())
        S_MAPE.append(lista.MAPE.std())
    
    import numpy as np
    M_MAE = np.array(M_MAE)
    S_MAE = np.array(S_MAE)

    M_RMSE = np.array(M_RMSE)
    S_RMSE = np.array(S_RMSE)

    M_MAPE = np.array(M_MAPE)
    S_MAPE = np.array(S_MAPE)

    import matplotlib.pyplot as plt
    X = list(range(1,6))

    fig, ax = plt.subplots(figsize=(13.7, 8.27))

    plt.plot(X, M_RMSE, color='purple', marker='o', markersize=5, label='RMSE')
    plt.fill_between(X, M_RMSE + S_RMSE, M_RMSE - S_RMSE, alpha=0.15, color='purple')

    plt.plot(X, M_MAE, color='green', marker='o', markersize=5, label='MAE')
    plt.fill_between(X, M_MAE + S_MAE, M_MAE - S_MAE, alpha=0.15, color='green')

    plt.plot(X, M_MAPE, color='gold', marker='o', markersize=5, label='MAPE')
    plt.fill_between(X, M_MAPE + S_MAPE, M_MAPE - S_MAPE, alpha=0.15, color='gold')

    plt.grid()
    plt.title("Erro médio dos 100 treinamentos de cada arquitetura MLP", fontsize=18)
    plt.xlabel('Número de neurônios', fontsize=18)
    plt.ylabel('Erro', fontsize=18)
    plt.legend(fontsize=18)

In [ ]:
baseline('Temp')

In [ ]:
baseline('OD')

In [ ]:
baseline('PH')

In [ ]:
treino_neuronios('Temp', 'neuronios_Temp.csv')

In [ ]:
treino_neuronios('OD', 'neuronios_OD.csv')

In [ ]:
treino_neuronios('PH', 'neuronios_PH.csv')

In [ ]:
analise_neuronios('neuronios_Temp.csv')


In [ ]:
analise_neuronios('neuronios_OD.csv')

In [ ]:
analise_neuronios('neuronios_PH.csv')

In [ ]:
treino_camadas('Temp', 140, 'camadas_Temp.csv')

In [ ]:
treino_camadas('OD', 180, 'camadas_OD.csv')

In [ ]:
treino_camadas('PH', 180, 'camadas_PH.csv')

In [ ]:
analise_camadas('camadas_Temp.csv')

In [ ]:
analise_camadas('camadas_OD.csv')

In [ ]:
analise_camadas('camadas_PH.csv')

In [ ]:
treino_features(['Temp', 'TEMP_40', 'TEMP_50', 'TEMP_60'], 'Temp', (180), 'derivacao_Temp.csv')

In [ ]:
treino_features(['OD', 'OD_40', 'OD_50', 'OD_60'], 'OD', (180,180,180,180,180), 'derivacao_OD.csv')

In [ ]:
treino_features(['PH', 'PH_40', 'PH_50', 'PH_60'], 'PH', (180,180,180), 'derivacao_PH.csv')

In [ ]:
analise_derivacao('derivacao_Temp.csv')

In [ ]:
analise_derivacao('derivacao_OD.csv')

In [ ]:
analise_derivacao('derivacao_PH.csv')

In [ ]:
treino_janelas(['Temp'], 'Temp', (180), 'janelas_Temp.csv')

In [ ]:
treino_janelas(['OD'], 'OD', (180,180,180,180,180), 'janelas_OD.csv')

In [ ]:
treino_janelas(['PH', 'PH_40', 'PH_50', 'PH_60'], 'PH', (180,180,180), 'janelas_PH.csv')

In [ ]:
analise_janelas('janelas_Temp.csv')

In [ ]:
analise_janelas('janelas_OD.csv')

In [ ]:
analise_janelas('janelas_PH.csv')

In [ ]:
dfbox = pd.read_csv('neuronios_PH.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [dfbox.loc[dfbox.neurons == 20].RMSE, 
        dfbox.loc[dfbox.neurons == 40].RMSE,
        dfbox.loc[dfbox.neurons == 60].RMSE,
        dfbox.loc[dfbox.neurons == 80].RMSE,
        dfbox.loc[dfbox.neurons == 100].RMSE,
        dfbox.loc[dfbox.neurons == 120].RMSE,
        dfbox.loc[dfbox.neurons == 140].RMSE,
        dfbox.loc[dfbox.neurons == 160].RMSE,
        dfbox.loc[dfbox.neurons == 180].RMSE,
        dfbox.loc[dfbox.neurons == 200].RMSE,
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [20,40,60,80,100,120,140,160,180,200])
plt.xlabel('Quantidade de neurônios')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de neurônios na camada oculta')
plt.show()

In [ ]:
dfbox = pd.read_csv('neuronios_PH.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [dfbox.loc[dfbox.neurons == 20].MAE, 
        dfbox.loc[dfbox.neurons == 40].MAE,
        dfbox.loc[dfbox.neurons == 60].MAE,
        dfbox.loc[dfbox.neurons == 80].MAE,
        dfbox.loc[dfbox.neurons == 100].MAE,
        dfbox.loc[dfbox.neurons == 120].MAE,
        dfbox.loc[dfbox.neurons == 140].MAE,
        dfbox.loc[dfbox.neurons == 160].MAE,
        dfbox.loc[dfbox.neurons == 180].MAE,
        dfbox.loc[dfbox.neurons == 200].MAE,
       ]

plt.boxplot(MAE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [20,40,60,80,100,120,140,160,180,200])
plt.xlabel('Quantidade de neurônios')
plt.ylabel('MAE')
plt.title('Avaliação da variação de neurônios na camada oculta')
plt.show()

In [ ]:
dfbox = pd.read_csv('neuronios_PH.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAPE = [dfbox.loc[dfbox.neurons == 20].MAPE, 
        dfbox.loc[dfbox.neurons == 40].MAPE,
        dfbox.loc[dfbox.neurons == 60].MAPE,
        dfbox.loc[dfbox.neurons == 80].MAPE,
        dfbox.loc[dfbox.neurons == 100].MAPE,
        dfbox.loc[dfbox.neurons == 120].MAPE,
        dfbox.loc[dfbox.neurons == 140].MAPE,
        dfbox.loc[dfbox.neurons == 160].MAPE,
        dfbox.loc[dfbox.neurons == 180].MAPE,
        dfbox.loc[dfbox.neurons == 200].MAPE,
       ]

plt.boxplot(MAPE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [20,40,60,80,100,120,140,160,180,200])
plt.xlabel('Quantidade de neurônios')
plt.ylabel('MAPE')
plt.title('Avaliação da variação de neurônios na camada oculta')
plt.show()

In [ ]:
dfbox = pd.read_csv('neuronios_Temp.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [dfbox.loc[dfbox.neurons == 20].RMSE, 
        dfbox.loc[dfbox.neurons == 40].RMSE,
        dfbox.loc[dfbox.neurons == 60].RMSE,
        dfbox.loc[dfbox.neurons == 80].RMSE,
        dfbox.loc[dfbox.neurons == 100].RMSE,
        dfbox.loc[dfbox.neurons == 120].RMSE,
        dfbox.loc[dfbox.neurons == 140].RMSE,
        dfbox.loc[dfbox.neurons == 160].RMSE,
        dfbox.loc[dfbox.neurons == 180].RMSE,
        dfbox.loc[dfbox.neurons == 200].RMSE,
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [20,40,60,80,100,120,140,160,180,200])
plt.xlabel('Quantidade de neurônios')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de neurônios na camada oculta')
plt.show()

In [ ]:
dfbox = pd.read_csv('neuronios_Temp.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [dfbox.loc[dfbox.neurons == 20].MAE, 
        dfbox.loc[dfbox.neurons == 40].MAE,
        dfbox.loc[dfbox.neurons == 60].MAE,
        dfbox.loc[dfbox.neurons == 80].MAE,
        dfbox.loc[dfbox.neurons == 100].MAE,
        dfbox.loc[dfbox.neurons == 120].MAE,
        dfbox.loc[dfbox.neurons == 140].MAE,
        dfbox.loc[dfbox.neurons == 160].MAE,
        dfbox.loc[dfbox.neurons == 180].MAE,
        dfbox.loc[dfbox.neurons == 200].MAE,
       ]

plt.boxplot(MAE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [20,40,60,80,100,120,140,160,180,200])
plt.xlabel('Quantidade de neurônios')
plt.ylabel('MAE')
plt.title('Avaliação da variação de neurônios na camada oculta')
plt.show()

In [ ]:
dfbox = pd.read_csv('neuronios_Temp.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAPE = [dfbox.loc[dfbox.neurons == 20].MAPE, 
        dfbox.loc[dfbox.neurons == 40].MAPE,
        dfbox.loc[dfbox.neurons == 60].MAPE,
        dfbox.loc[dfbox.neurons == 80].MAPE,
        dfbox.loc[dfbox.neurons == 100].MAPE,
        dfbox.loc[dfbox.neurons == 120].MAPE,
        dfbox.loc[dfbox.neurons == 140].MAPE,
        dfbox.loc[dfbox.neurons == 160].MAPE,
        dfbox.loc[dfbox.neurons == 180].MAPE,
        dfbox.loc[dfbox.neurons == 200].MAPE,
       ]

plt.boxplot(MAPE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [20,40,60,80,100,120,140,160,180,200])
plt.xlabel('Quantidade de neurônios')
plt.ylabel('MAPE')
plt.title('Avaliação da variação de neurônios na camada oculta')
plt.show()

In [ ]:
dfbox = pd.read_csv('neuronios_OD.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [dfbox.loc[dfbox.neurons == 20].RMSE, 
        dfbox.loc[dfbox.neurons == 40].RMSE,
        dfbox.loc[dfbox.neurons == 60].RMSE,
        dfbox.loc[dfbox.neurons == 80].RMSE,
        dfbox.loc[dfbox.neurons == 100].RMSE,
        dfbox.loc[dfbox.neurons == 120].RMSE,
        dfbox.loc[dfbox.neurons == 140].RMSE,
        dfbox.loc[dfbox.neurons == 160].RMSE,
        dfbox.loc[dfbox.neurons == 180].RMSE,
        dfbox.loc[dfbox.neurons == 200].RMSE,
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [20,40,60,80,100,120,140,160,180,200])
plt.xlabel('Quantidade de neurônios')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de neurônios na camada oculta')
plt.show()

In [ ]:
dfbox = pd.read_csv('neuronios_OD.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [dfbox.loc[dfbox.neurons == 20].MAE, 
        dfbox.loc[dfbox.neurons == 40].MAE,
        dfbox.loc[dfbox.neurons == 60].MAE,
        dfbox.loc[dfbox.neurons == 80].MAE,
        dfbox.loc[dfbox.neurons == 100].MAE,
        dfbox.loc[dfbox.neurons == 120].MAE,
        dfbox.loc[dfbox.neurons == 140].MAE,
        dfbox.loc[dfbox.neurons == 160].MAE,
        dfbox.loc[dfbox.neurons == 180].MAE,
        dfbox.loc[dfbox.neurons == 200].MAE,
       ]

plt.boxplot(MAE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [20,40,60,80,100,120,140,160,180,200])
plt.xlabel('Quantidade de neurônios')
plt.ylabel('MAE')
plt.title('Avaliação da variação de neurônios na camada oculta')
plt.show()

In [ ]:
dfbox = pd.read_csv('neuronios_OD.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAPE = [dfbox.loc[dfbox.neurons == 20].MAPE, 
        dfbox.loc[dfbox.neurons == 40].MAPE,
        dfbox.loc[dfbox.neurons == 60].MAPE,
        dfbox.loc[dfbox.neurons == 80].MAPE,
        dfbox.loc[dfbox.neurons == 100].MAPE,
        dfbox.loc[dfbox.neurons == 120].MAPE,
        dfbox.loc[dfbox.neurons == 140].MAPE,
        dfbox.loc[dfbox.neurons == 160].MAPE,
        dfbox.loc[dfbox.neurons == 180].MAPE,
        dfbox.loc[dfbox.neurons == 200].MAPE,
       ]

plt.boxplot(MAPE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [20,40,60,80,100,120,140,160,180,200])
plt.xlabel('Quantidade de neurônios')
plt.ylabel('MAPE')
plt.title('Avaliação da variação de neurônios na camada oculta')
plt.show()

In [ ]:
dfbox = pd.read_csv('camadas_PH.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [
        dfbox.loc[dfbox.layers == 2].RMSE,
        dfbox.loc[dfbox.layers == 3].RMSE,
        dfbox.loc[dfbox.layers == 4].RMSE,
        dfbox.loc[dfbox.layers == 5].RMSE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4], [2,3,4,5])
plt.xlabel('Quantidade de camadas ocultas')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de camadas ocultas')
plt.show()

In [ ]:
dfbox = pd.read_csv('camadas_PH.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [
        dfbox.loc[dfbox.layers == 2].MAE,
        dfbox.loc[dfbox.layers == 3].MAE,
        dfbox.loc[dfbox.layers == 4].MAE,
        dfbox.loc[dfbox.layers == 5].MAE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4], [2,3,4,5])
plt.xlabel('Quantidade de camadas ocultas')
plt.ylabel('MAE')
plt.title('Avaliação da variação de camadas ocultas')
plt.show()

In [ ]:
dfbox = pd.read_csv('camadas_OD.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [
        dfbox.loc[dfbox.layers == 2].RMSE,
        dfbox.loc[dfbox.layers == 3].RMSE,
        dfbox.loc[dfbox.layers == 4].RMSE,
        dfbox.loc[dfbox.layers == 5].RMSE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4], [2,3,4,5])
plt.xlabel('Quantidade de camadas ocultas')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de camadas ocultas')
plt.show()

In [ ]:
dfbox = pd.read_csv('camadas_OD.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [
        dfbox.loc[dfbox.layers == 2].MAE,
        dfbox.loc[dfbox.layers == 3].MAE,
        dfbox.loc[dfbox.layers == 4].MAE,
        dfbox.loc[dfbox.layers == 5].MAE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4], [2,3,4,5])
plt.xlabel('Quantidade de camadas ocultas')
plt.ylabel('MAE')
plt.title('Avaliação da variação de camadas ocultas')
plt.show()

In [ ]:
dfbox = pd.read_csv('camadas_Temp.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [
        dfbox.loc[dfbox.layers == 2].RMSE,
        dfbox.loc[dfbox.layers == 3].RMSE,
        dfbox.loc[dfbox.layers == 4].RMSE,
        dfbox.loc[dfbox.layers == 5].RMSE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4], [2,3,4,5])
plt.xlabel('Quantidade de camadas ocultas')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de camadas ocultas')
plt.show()

In [ ]:
dfbox = pd.read_csv('camadas_Temp.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [
        dfbox.loc[dfbox.layers == 2].MAE,
        dfbox.loc[dfbox.layers == 3].MAE,
        dfbox.loc[dfbox.layers == 4].MAE,
        dfbox.loc[dfbox.layers == 5].MAE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xticks([1, 2, 3, 4], [2,3,4,5])
plt.xlabel('Quantidade de camadas ocultas')
plt.ylabel('MAE')
plt.title('Avaliação da variação de camadas ocultas')
plt.show()

In [ ]:
dfbox = pd.read_csv('derivacao_Temp.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [
        dfbox.loc[dfbox.n_features == 1].RMSE,
        dfbox.loc[dfbox.n_features == 2].RMSE,
        dfbox.loc[dfbox.n_features == 3].RMSE,
        dfbox.loc[dfbox.n_features == 4].RMSE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de features')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de features')
plt.show()

In [ ]:
dfbox = pd.read_csv('derivacao_Temp.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [
        dfbox.loc[dfbox.n_features == 1].MAE,
        dfbox.loc[dfbox.n_features == 2].MAE,
        dfbox.loc[dfbox.n_features == 3].MAE,
        dfbox.loc[dfbox.n_features == 4].MAE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de features')
plt.ylabel('MAE')
plt.title('Avaliação da variação de features')
plt.show()

In [ ]:
 dfbox = pd.read_csv('derivacao_OD.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [
        dfbox.loc[dfbox.n_features == 1].RMSE,
        dfbox.loc[dfbox.n_features == 2].RMSE,
        dfbox.loc[dfbox.n_features == 3].RMSE,
        dfbox.loc[dfbox.n_features == 4].RMSE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de features')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de features')
plt.show()

In [ ]:
dfbox = pd.read_csv('derivacao_OD.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [
        dfbox.loc[dfbox.n_features == 1].MAE,
        dfbox.loc[dfbox.n_features == 2].MAE,
        dfbox.loc[dfbox.n_features == 3].MAE,
        dfbox.loc[dfbox.n_features == 4].MAE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de features')
plt.ylabel('MAE')
plt.title('Avaliação da variação de features')
plt.show()

In [ ]:
 dfbox = pd.read_csv('derivacao_PH.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [
        dfbox.loc[dfbox.n_features == 1].RMSE,
        dfbox.loc[dfbox.n_features == 2].RMSE,
        dfbox.loc[dfbox.n_features == 3].RMSE,
        dfbox.loc[dfbox.n_features == 4].RMSE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de features')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de features')
plt.show()

In [ ]:
dfbox = pd.read_csv('derivacao_PH.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [
        dfbox.loc[dfbox.n_features == 1].MAE,
        dfbox.loc[dfbox.n_features == 2].MAE,
        dfbox.loc[dfbox.n_features == 3].MAE,
        dfbox.loc[dfbox.n_features == 4].MAE
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de features')
plt.ylabel('MAE')
plt.title('Avaliação da variação de features')
plt.show()

In [ ]:
 dfbox = pd.read_csv('janelas_Temp.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [
        dfbox.loc[dfbox.windows == 1].RMSE,
        dfbox.loc[dfbox.windows == 2].RMSE,
        dfbox.loc[dfbox.windows == 3].RMSE,
        dfbox.loc[dfbox.windows == 4].RMSE,
        dfbox.loc[dfbox.windows == 5].RMSE,
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de janelas')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de janelas')
plt.show()

In [ ]:
 dfbox = pd.read_csv('janelas_Temp.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [
        dfbox.loc[dfbox.windows == 1].MAE,
        dfbox.loc[dfbox.windows == 2].MAE,
        dfbox.loc[dfbox.windows == 3].MAE,
        dfbox.loc[dfbox.windows == 4].MAE,
        dfbox.loc[dfbox.windows == 5].MAE,
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de janelas')
plt.ylabel('MAE')
plt.title('Avaliação da variação de janelas')
plt.show()

In [ ]:
 dfbox = pd.read_csv('janelas_OD.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [
        dfbox.loc[dfbox.windows == 1].RMSE,
        dfbox.loc[dfbox.windows == 2].RMSE,
        dfbox.loc[dfbox.windows == 3].RMSE,
        dfbox.loc[dfbox.windows == 4].RMSE,
        dfbox.loc[dfbox.windows == 5].RMSE,
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de janelas')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de janelas')
plt.show()

In [ ]:
 dfbox = pd.read_csv('janelas_OD.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [
        dfbox.loc[dfbox.windows == 1].MAE,
        dfbox.loc[dfbox.windows == 2].MAE,
        dfbox.loc[dfbox.windows == 3].MAE,
        dfbox.loc[dfbox.windows == 4].MAE,
        dfbox.loc[dfbox.windows == 5].MAE,
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de janelas')
plt.ylabel('MAE')
plt.title('Avaliação da variação de janelas')
plt.show()

In [ ]:
 dfbox = pd.read_csv('janelas_PH.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

RMSE = [
        dfbox.loc[dfbox.windows == 1].RMSE,
        dfbox.loc[dfbox.windows == 2].RMSE,
        dfbox.loc[dfbox.windows == 3].RMSE,
        dfbox.loc[dfbox.windows == 4].RMSE,
        dfbox.loc[dfbox.windows == 5].RMSE,
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de janelas')
plt.ylabel('RMSE')
plt.title('Avaliação da variação de janelas')
plt.show()

In [ ]:
 dfbox = pd.read_csv('janelas_PH.csv')
dfbox = dfbox.drop(columns=['Unnamed: 0'])

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))

MAE = [
        dfbox.loc[dfbox.windows == 1].MAE,
        dfbox.loc[dfbox.windows == 2].MAE,
        dfbox.loc[dfbox.windows == 3].MAE,
        dfbox.loc[dfbox.windows == 4].MAE,
        dfbox.loc[dfbox.windows == 5].MAE,
       ]

plt.boxplot(RMSE, medianprops={"linewidth": 2})
plt.xlabel('Quantidade de janelas')
plt.ylabel('MAE')
plt.title('Avaliação da variação de janelas')
plt.show()

## Trieno Temperatura

In [ ]:
column = ['Temp']
column_y = 'Temp'
window = 1
train_size = 0.7
shift_prediction = 6
result = pd.read_csv('dataset_preprocessado.csv', sep=',')
result = result.drop(['Unnamed: 0'], axis=1)
result
data = window_all(result[column].reset_index(drop=True), window, column_y)
data = data.dropna()

shift_prediction *= -1
X = data.drop(['y'], axis=1)
y = data['y'].shift(periods=shift_prediction)
data = pd.concat([X, y], axis=1)
data = data[:shift_prediction]
data = data.astype('float64')
X_train, X_test, y_train, y_test = train_test_split(data, train_size)

data

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(180), max_iter=1000, solver='adam',
                         validation_fraction=0.1,  verbose=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
M = metrics(y_test, y_pred)

In [ ]:
M

In [ ]:
erro_mlp = abs(y_test - y_pred)
erro_baseline = baseline_error('Temp')

In [ ]:
from scipy import stats
stats.ks_2samp(erro_mlp, erro_baseline)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))
sns.set_style('whitegrid')

sns.kdeplot(np.array(erro_mlp), bw=0.5, label='MLP')
sns.kdeplot(np.array(erro_baseline), bw=0.5, label='baseline')
plt.xlabel('Erro')
plt.ylabel('Densidade')
plt.title('Densidade do erro das predições de temperatura')
plt.legend()
plt.show()

## Trieno OD

In [ ]:
column = ['OD']
column_y = 'OD'
window = 1
train_size = 0.7
shift_prediction = 6
result = pd.read_csv('dataset_preprocessado.csv', sep=',')
result = result.drop(['Unnamed: 0'], axis=1)
result
data = window_all(result[column].reset_index(drop=True), window, column_y)
data = data.dropna()

shift_prediction *= -1
X = data.drop(['y'], axis=1)
y = data['y'].shift(periods=shift_prediction)
data = pd.concat([X, y], axis=1)
data = data[:shift_prediction]
data = data.astype('float64')
X_train, X_test, y_train, y_test = train_test_split(data, train_size)

data

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(180,180,180,180,180), max_iter=1000, solver='adam',
                         validation_fraction=0.1,  verbose=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
M = metrics(y_test, y_pred)

In [ ]:
M

In [ ]:
erro_mlp = y_test - y_pred
erro_baseline = baseline_error('OD')

In [ ]:
from scipy import stats
stats.ks_2samp(erro_mlp, erro_baseline)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))
sns.set_style('whitegrid')

sns.kdeplot(np.array(erro_mlp), bw=0.5, label='MLP')
sns.kdeplot(np.array(erro_baseline), bw=0.5, label='baseline')
plt.xlabel('Erro')
plt.ylabel('Densidade')
plt.title('Densidade do erro das predições de OD')
plt.legend()
plt.show()

## Trieno PH

In [ ]:
column = ['PH', 'PH_40', 'PH_50', 'PH_60']
column_y = 'PH'
window = 4
train_size = 0.7
shift_prediction = 6
result = pd.read_csv('dataset_preprocessado.csv', sep=',')
result = result.drop(['Unnamed: 0'], axis=1)
result
data = window_all(result[column].reset_index(drop=True), window, column_y)
data = data.dropna()

shift_prediction *= -1
X = data.drop(['y'], axis=1)
y = data['y'].shift(periods=shift_prediction)
data = pd.concat([X, y], axis=1)
data = data[:shift_prediction]
data = data.astype('float64')
X_train, X_test, y_train, y_test = train_test_split(data, train_size)

data

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(180,180,180), max_iter=1000, solver='adam',
                         validation_fraction=0.1,  verbose=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
M = metrics(y_test, y_pred)

In [ ]:
M

In [ ]:
erro_mlp = abs(y_test - y_pred)
erro_baseline = baseline_error('PH')

In [ ]:
from scipy import stats
stats.ks_2samp(erro_mlp, erro_baseline)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(13.7, 8.27))
sns.set_style('whitegrid')

sns.kdeplot(np.array(erro_mlp), bw=0.5, label='MLP')
sns.kdeplot(np.array(erro_baseline), bw=0.5, label='baseline')
plt.xlabel('Erro')
plt.ylabel('Densidade')
plt.title('Densidade do erro das predições de PH')
plt.legend()
plt.show()

## Detecção de anomalia

# Temperatura

In [ ]:
column = ['Temp']
column_y = 'Temp'
window = 1
train_size = 0.7
shift_prediction = 6
result = pd.read_csv('dataset_preprocessado.csv', sep=',')
result = result.drop(['Unnamed: 0'], axis=1)
result
data = window_all(result[column].reset_index(drop=True), window, column_y)
data = data.dropna()

shift_prediction *= -1
X = data.drop(['y'], axis=1)
y = data['y'].shift(periods=shift_prediction)
data = pd.concat([X, y], axis=1)
data = data[:shift_prediction]
data = data.astype('float64')
X_train, X_test, y_train, y_test = train_test_split(data, train_size)

data

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(180), max_iter=1000, solver='adam',
                         validation_fraction=0.1,  verbose=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
M = metrics(y_test, y_pred)

In [ ]:
import numpy as np
anomaly = pd.DataFrame()
anomaly['y'] = np.concatenate([y_train, y_pred])

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

outliers_fraction = float(.1)
scaler = StandardScaler()
np_scaled = scaler.fit_transform(anomaly.y.values.reshape(-1,1))
dt = pd.DataFrame(np_scaled)

model = IsolationForest(n_estimators=100, contamination=outliers_fraction)
model.fit(dt[:len(y_train)])

import numpy as np

append = [1]*len(y_train)
x = model.predict(dt[len(y_train):])
prd = np.concatenate([append, x])

data['anomaly'] = prd    
a = data.loc[data['anomaly'] == -1, ['y']]

plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(figsize=(12,8))

ax.plot(data[['y']][:len(y_train)], color='#069AF3', linewidth=3.0, label='Valores do sensor')
ax.plot(data[['y']][len(y_train):], color='#F97306', linewidth=3.0, label='Valores da predição')
ax.scatter(a.index, a['y'], s= 100.0, color='black', label='Anomalia')
plt.legend()

# OD

In [ ]:
column = ['OD']
column_y = 'OD'
window = 1
train_size = 0.7
shift_prediction = 6
result = pd.read_csv('dataset_preprocessado.csv', sep=',')
result = result.drop(['Unnamed: 0'], axis=1)
result
data = window_all(result[column].reset_index(drop=True), window, column_y)
data = data.dropna()

shift_prediction *= -1
X = data.drop(['y'], axis=1)
y = data['y'].shift(periods=shift_prediction)
data = pd.concat([X, y], axis=1)
data = data[:shift_prediction]
data = data.astype('float64')
X_train, X_test, y_train, y_test = train_test_split(data, train_size)

data

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(180,180,180,180,180), max_iter=1000, solver='adam',
                         validation_fraction=0.1,  verbose=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
M = metrics(y_test, y_pred)

In [ ]:
import numpy as np
anomaly = pd.DataFrame()
anomaly['y'] = np.concatenate([y_train, y_pred])

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

outliers_fraction = float(.1)
scaler = StandardScaler()
np_scaled = scaler.fit_transform(anomaly.y.values.reshape(-1,1))
dt = pd.DataFrame(np_scaled)

model = IsolationForest(n_estimators=100, contamination=outliers_fraction)
model.fit(dt[:len(y_train)])

import numpy as np

append = [1]*len(y_train)
x = model.predict(dt[len(y_train):])
prd = np.concatenate([append, x])

data['anomaly'] = prd    
a = data.loc[data['anomaly'] == -1, ['y']]

plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(figsize=(12,8))

ax.plot(data[['y']][:len(y_train)], color='#069AF3', linewidth=3.0, label='Valores do sensor')
ax.plot(data[['y']][len(y_train):], color='#F97306', linewidth=3.0, label='Valores da predição')
ax.scatter(a.index, a['y'], s= 100.0, color='black', label='Anomalia')
plt.legend()

# PH

In [ ]:
column = ['PH', 'PH_40', 'PH_50', 'PH_60']
column_y = 'PH'
window = 4
train_size = 0.7
shift_prediction = 6
result = pd.read_csv('dataset_preprocessado.csv', sep=',')
result = result.drop(['Unnamed: 0'], axis=1)
result
data = window_all(result[column].reset_index(drop=True), window, column_y)
data = data.dropna()

shift_prediction *= -1
X = data.drop(['y'], axis=1)
y = data['y'].shift(periods=shift_prediction)
data = pd.concat([X, y], axis=1)
data = data[:shift_prediction]
data = data.astype('float64')
X_train, X_test, y_train, y_test = train_test_split(data, train_size)

data

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(180,180,180), max_iter=1000, solver='adam',
                         validation_fraction=0.1,  verbose=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
M = metrics(y_test, y_pred)

In [ ]:
import numpy as np
anomaly = pd.DataFrame()
anomaly['y'] = np.concatenate([y_train, y_pred])

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

outliers_fraction = float(.1)
scaler = StandardScaler()
np_scaled = scaler.fit_transform(anomaly.y.values.reshape(-1,1))
dt = pd.DataFrame(np_scaled)

model = IsolationForest(n_estimators=100, contamination=outliers_fraction)
model.fit(dt[:len(y_train)])

import numpy as np

append = [1]*len(y_train)
x = model.predict(dt[len(y_train):])
prd = np.concatenate([append, x])

data['anomaly'] = prd    
a = data.loc[data['anomaly'] == -1, ['y']]

plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(figsize=(12,8))

ax.plot(data[['y']][:len(y_train)], color='#069AF3', linewidth=3.0, label='Valores do sensor')
ax.plot(data[['y']][len(y_train):], color='#F97306', linewidth=3.0, label='Valores da predição')
ax.scatter(a.index, a['y'], s= 100.0, color='black', label='Anomalia')
plt.legend()